In [1]:
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from tqdm import tqdm

In [87]:
def add_to_db(word,word_ka,word_fr,word_ru,word_kr,word_jp,word_pronunciation,word_type,word_gender,word_tag):
        try:
            connection = mysql.connector.connect(
                host='localhost',
                user='root',
                password='RandoDumbai18',
                database='cosmolingo'
            )
            cursor = connection.cursor()
            query_check = "SELECT COUNT(*) FROM words WHERE word = %s"
            cursor.execute(query_check, (word,))
            result = cursor.fetchone()
            if result[0] == 0:
                query_insert = "INSERT INTO words (word,word_ka,word_fr,word_ru,word_kr,word_jp,word_pronunciation,word_type,word_gender,word_tag) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(query_insert, (word,word_ka,word_fr,word_ru,word_kr,word_jp,word_pronunciation,word_type,word_gender,word_tag))
            connection.commit()
        except mysql.connector.Error as err:
            print(f"Error: {err}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

In [ ]:
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='RandoDumbai18',
    database='cosmolingo'
)

cursor = connection.cursor()
query_check = "SELECT * FROM words"
cursor.execute(query_check)
result = cursor.fetchone()
print(result)

[1, 'cat', 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 'мысық', 'кот', 'chat', '고양이', '', 'ch_a', 'n', 'm', 'animal', datetime.datetime(2025, 4, 22, 0, 0)]


In [21]:
add_to_db("test")

(1,)


In [94]:
connection = mysql.connector.connect(
                host='localhost',
                user='root',
                password='RandoDumbai18',
                database='cosmolingo'
)
cursor = connection.cursor()
query_check = "DELETE FROM words"
cursor.execute(query_check)
result = cursor.fetchone()
query_check = "ALTER TABLE words AUTO_INCREMENT = 1"
cursor.execute(query_check)
result = cursor.fetchone()

In [95]:
#query = "ALTER TABLE words AUTO_INCREMENT = 1;"

with open('words.txt') as f:
    words = f.read().split('\n')
    for word in words:
        word_meta = word.split(':')[0]
        if ';' in word_meta:
            word_tag = word_meta.split(';')[1]
            word_meta = word_meta.split(';')[0]
        else:
            word_tag = None
        if '-' not in word_meta:
            word_type = word_meta
            word_gender = None
        else:
            word_type = word_meta.split('-')[0]
            word_gender = word_meta.split('-')[1]
        word_translations = word.split(':')[1:]
        if '(' in word_translations[2]:
            word_pronunciation = word_translations[2].split('(')[1].split(')')[0]
            word_translations[2] = word_translations[2].split('(')[0]
        else:
            word_pronunciation = None
        word_ka = word_translations[0]
        word_en = word_translations[1]
        word_translations[0] = word_en
        word_translations[1] = word_ka
        
        add_to_db(*word_translations,word_pronunciation,word_type,word_gender,word_tag)

In [68]:
sentences = []

In [83]:
# Set up Firefox options
firefox_options = Options()
firefox_options.add_argument("--headless")  # Ensure GUI is off
firefox_options.add_argument("--no-sandbox")
firefox_options.add_argument("--disable-dev-shm-usage")

# Set up the webdriver
driver = webdriver.Firefox(options=firefox_options)

for i in tqdm(range(10_000)):
    # Open the page
    driver.get("https://duome.eu/practice/en/fr")

    # Get the content of the .fs20 class
    content = driver.find_element(By.CLASS_NAME, "fs20").get_attribute('outerHTML')

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Remove the <b> tag
    for b_tag in soup.find_all('b'):
        b_tag.unwrap()

    # Remove all attributes from <a> tags
    for a_tag in soup.find_all('a'):
        a_tag.attrs = {}

    soup = str(soup)
    soup = soup.replace('<a>','<')
    soup = soup.replace('</a>','>')

    if soup not in sentences:
        sentences.append(soup)
        tqdm.write(str(len(sentences)))

# Close the driver
driver.quit()

sentences

 12%|█▏        | 1205/10000 [04:05<27:29,  5.33it/s] 

4343


 14%|█▍        | 1442/10000 [04:54<29:15,  4.88it/s]

4344


 16%|█▌        | 1581/10000 [05:23<30:30,  4.60it/s]

...

4356


 95%|█████████▌| 9515/10000 [33:13<02:36,  3.09it/s]

4357


100%|██████████| 10000/10000 [34:53<00:00,  4.78it/s]


['Je suis un <citoyen> français.',
 'Êtes-vous contente de votre <poids> ?',
 'La ville a une population de deux <millions> de personnes.',
 'La largeur de la porte est de quatre-vingts <centimètres>.',
 "Elle a écrit un <article> pour le journal de l'université.",
 "La chaleur est une forme d'<énergie>.",
 'Nos maisons ont des <toits> gris.',
 'La nourriture est pour les <rois>.',
 "C'est <impossible>.",
 'Il a oublié son passeport et il est coincé à la <frontière> !',
 'Le chien <respecte> le loup.',
 'Il a un <million> de pièces.',
 "Il n'est pas <fatigué>.",
 'Es-tu <intéressé> ?',
 "On fait ce <qu'>on peut.",
 'Tu connais cet <auteur> ?',
 'Les mercredis, <normalement>.',
 'une <matière>',
 "J'ai un <ennemi>.",
 'Le garçon et la fille sont <des> enfants.',
 "J'ai eu beaucoup de <difficultés> à me faire comprendre.",
 'Nous avons tous des <droits> et des devoirs.',
 'Elle a une bonne <odeur>.',
 'Et si tu ne trouvais pas ce <document> ?',
 'Le <thème> du jeu est toujours pour des e

In [80]:
len(sentences)

4339

In [84]:
with open('sentences.txt', 'w') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

In [85]:
sentences

['Je suis un <citoyen> français.',
 'Êtes-vous contente de votre <poids> ?',
 'La ville a une population de deux <millions> de personnes.',
 'La largeur de la porte est de quatre-vingts <centimètres>.',
 "Elle a écrit un <article> pour le journal de l'université.",
 "La chaleur est une forme d'<énergie>.",
 'Nos maisons ont des <toits> gris.',
 'La nourriture est pour les <rois>.',
 "C'est <impossible>.",
 'Il a oublié son passeport et il est coincé à la <frontière> !',
 'Le chien <respecte> le loup.',
 'Il a un <million> de pièces.',
 "Il n'est pas <fatigué>.",
 'Es-tu <intéressé> ?',
 "On fait ce <qu'>on peut.",
 'Tu connais cet <auteur> ?',
 'Les mercredis, <normalement>.',
 'une <matière>',
 "J'ai un <ennemi>.",
 'Le garçon et la fille sont <des> enfants.',
 "J'ai eu beaucoup de <difficultés> à me faire comprendre.",
 'Nous avons tous des <droits> et des devoirs.',
 'Elle a une bonne <odeur>.',
 'Et si tu ne trouvais pas ce <document> ?',
 'Le <thème> du jeu est toujours pour des e